In [2]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [7]:
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [8]:
mnist.train.labels.shape

(55000, 10)

In [10]:
mnist.train.images.shape

(55000, 784)

In [11]:
x = tf.placeholder(tf.float32, [None, 784], name='x_placeholder')
y = tf.placeholder(tf.int32, [None, 10], name='y_placeholder')

In [12]:
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [20]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='w1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='w2'),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='w')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='bias')
}

In [23]:
def multilayer_perception(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'], name='fc_1')
    layer_1 = tf.nn.relu(layer_1, name='relu_1')
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'], name='fc_2')
    layer_2 = tf.nn.relu(layer_2, name='relu_2')
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'], name='fc_3')
    return out_layer


In [24]:
pred = multilayer_perception(x, weights, biases)

In [28]:
learning_rate=0.001
loss_all = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y,name='cross_entropy')
loss = tf.reduce_mean(loss_all, name='avg_loss')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [29]:
init = tf.global_variables_initializer()

In [30]:
training_epoches=15
batch_size=128
display_step = 1

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./graphs/multilayer_perception', sess.graph)
    for epoch in range(training_epoches):
        avg_loss = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, l = sess.run([optimizer, loss], feed_dict={x:batch_x, y: batch_y})
            avg_loss += l
        avg_loss /= total_batch
        print("epoch: %s, avg_loss: %s"%(epoch, avg_loss))
    correct_predictions = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, 'float'))
    print('acc: %s'%acc.eval({x:mnist.test.images, y: mnist.test.labels}))
    writer.close()

epoch: 0, avg_loss: 190.5244200290778
epoch: 1, avg_loss: 48.36861771581334
